<a href="https://colab.research.google.com/github/acarbn/portfolio/blob/main/ElasticNetworkModels/GNM/GNM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.PDBList import PDBList
import plotly.graph_objects as go

def GNM(fname1, mode_set, chainID):
    rcut_gnm = 7.0
    gamma = 1.0  # spring constant

    # Read PDB file
    parser = PDBParser(QUIET=True)
    try:
        pdbl = PDBList()
        pdbl.retrieve_pdb_file(fname1, pdir='.', file_format='pdb')
        structure = parser.get_structure('protein', f"pdb{fname1.lower()}.ent")
    except:
        print("PDB file not found")
        return

    coords = []
    chain_start_end = {}  # To store the residue range for each chain
    residue_counter = 0  # To keep track of the global residue number across all chains

    # Loop through all chains in the structure and check if they are in chainID
    for model in structure:
        for chain in model:
            if chain.get_id() in chainID:  # Check if the chain ID is in the provided chainID string
                chain_start_end[chain.get_id()] = (residue_counter, None)
                for residue in chain:
                    if 'CA' in residue:
                        atom = residue['CA']
                        if atom.altloc in ('A', ' ') or atom.altloc == '':
                            coords.append(atom.coord)
                            residue_counter += 1
                chain_start_end[chain.get_id()] = (chain_start_end[chain.get_id()][0], residue_counter)

    coords = np.array(coords)
    resnum = len(coords)

    # Kirchhoff matrix
    kirchhoff = np.zeros((resnum, resnum))
    for i in range(resnum):
        for j in range(i+1, resnum):
            dist = np.linalg.norm(coords[i] - coords[j])
            if 0.0001 < dist <= rcut_gnm:
                kirchhoff[i, j] = -1
                kirchhoff[j, i] = -1
    np.fill_diagonal(kirchhoff, -np.sum(kirchhoff, axis=1))

    # Eigen decomposition
    eigvals, eigvecs = np.linalg.eigh(kirchhoff)

    # Exclude zero (trivial) mode
    nonzero_modes = eigvals > 1e-6
    eigvals = eigvals[nonzero_modes]
    eigvecs = eigvecs[:, nonzero_modes]

    # Select desired modes (1-based index)
    MSF_all = np.zeros((resnum, len(mode_set)))
    for idx, mode_num in enumerate(mode_set):
        k = mode_num - 1  # 0-based indexing
        MSF_all[:, idx] = np.square(eigvecs[:, k]) / eigvals[k]

    # Mean MSF
    MSF_avg = np.mean(MSF_all, axis=1)
    MSF_avg /= np.trapz(MSF_avg)

    # Plot eigenvalues
    fig = go.Figure()
    fig.add_trace(go.Scatter(
    x=mode_set,
    y=[eigvals[mode - 1] for mode in mode_set],
    mode='lines+markers',  # Combine both lines and markers
    line=dict(color='black', width=3),  # Line styling
    marker=dict(color='black', size=6)  # Marker styling (adjust size as needed)
    ))
    fig.update_layout(
        width=500,
        height=500,
        xaxis_title='Mode Number',
        yaxis_title='Eigenvalue',
        font=dict(size=14),
        margin=dict(l=40, r=40, t=40, b=40),
    )
    fig.show()

    # Plot MSF with vertical lines separating chains
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=list(range(resnum)),
        y=MSF_avg,
        mode='lines',
        line=dict(color='black', width=3)
    ))

    # Add vertical lines for each chain's separation
    for chain, (start, end) in chain_start_end.items():
        if end is not None:
            fig2.add_shape(
                type="line",
                x0=end, y0=0, x1=end, y1=max(MSF_avg),
                line=dict(color="red", width=2, dash="dash")
            )

    # Adjust layout for MSF plot
    fig2.update_layout(
        width=800,
        height=800,
        xaxis_title='Residue Number',
        yaxis_title='MSF',
        font=dict(size=14),
        margin=dict(l=40, r=40, t=40, b=40),
    )
    fig2.show()

    return MSF_avg

# Example
PDBname="1L7V"
mode_set = list(range(1, 11)) # First 10 slow modes
chainID='ABCD'  # Specify chains A and B
msf = GNM(PDBname, mode_set, chainID)


Structure exists: './pdb1l7v.ent' 


<ipython-input-55-4b5c9edb5842>:67: DeprecationWarning:

`trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.

